In [1]:
!pip install -q transformers accelerate torch==2.1.0 bitsandbytes-cuda117 bitsandbytes huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.4 MB/s eta 0:00:00


In [2]:
#Generate response settings
maxTokenSettings = 2000
temperatureSettings = 0.9
top_pSettings = 0.9



In [3]:
from transformers.utils.quantization_config import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import accelerate
print(torch.version.cuda)

# Setup bitsandbytes quantization
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_double_nested_quant = True
load_in_8bit_fp32_cpu_offload = True

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_use_double_quant=use_double_nested_quant,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    load_in_8bit_fp32_cpu_offload=load_in_8bit_fp32_cpu_offload
)

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Python-hf")
model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-Python-hf", quantization_config=bnb_config, device_map="balanced_low_0") #codellama/CodeLlama-7b-hf

#Response Generation
def generate_smart_contract(input=""):
  input_ids = tokenizer(input, return_tensors="pt", truncation=True).input_ids.cuda()
  outputs = model.generate(input_ids=input_ids, max_new_tokens=maxTokenSettings, do_sample=True, top_p=top_pSettings, temperature=temperatureSettings, pad_token_id=tokenizer.eos_token_id)

  # Output the model result
  return (tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True, return_full_text=False)[0])

11.8


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
# Multiple generation
output = []

# The prompt
user_pr = input("Write your prompt: ") #"""Please write a full solidity smart contract to create a crypto token called "SagaCoin". The purpose of the smart contract is to allow the token holders to deposit, withdraw, and work with their tokens. using pragma solidity 0.8.0;"""

try:
  amountGen = int(input("Amount of generation: "))
except ValueError:
  print("\n> That is not a number!")

theprompt = f"<<SYS>> This is a code generation. <<SYS>> {user_pr}"
for i in range(amountGen):
  output.append(generate_smart_contract(theprompt))

#Export
import pandas as pd
df = pd.DataFrame({"Source Code": output})

file_name_set = input("File name(without .csv): ")
filesetname = file_name_set + ".csv"
df.to_csv(filesetname, sep='\t', encoding='utf-8', index=False)

Write your prompt: make a smart contract
Amount of generation: 1
File name(without .csv): nice


In [ ]:
# Single generation
print(generate_smart_contract(theprompt))